In [86]:
# !unzip test_all.zip

In [87]:
# !unzip train.zip

In [88]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.optim as optim

In [89]:
# torch.cuda.set_device(0)
device = torch.device("cpu")

In [90]:
torch.manual_seed(42)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [91]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomVerticalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.RandomErasing(0.4),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

In [92]:
full_dataset = torchvision.datasets.ImageFolder("train", transform=transform)


In [93]:
classes = full_dataset.classes

In [94]:
train_part = int(0.8 * len(full_dataset))
train_size = int(0.8 * train_part)
validation_size = train_part - train_size
test_size = len(full_dataset) - train_part
train_set, validation_set, test_set = torch.utils.data.random_split(full_dataset, [train_size, validation_size, test_size])

In [95]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [96]:
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [97]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [98]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=5, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(24, 128, 5)
        self.pool2 = nn.MaxPool2d(2)  ### obrazki sie zmniejszaja, dopytac na kons
        self.fc1 = nn.Linear(128 * 13 * 13, 1800) ### dopytac o to 16 x 5 x 5
        self.fc2 = nn.Linear(1800, 300)
        self.fc3 = nn.Linear(300, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [99]:
net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 24, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(24, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=21632, out_features=1800, bias=True)
  (fc2): Linear(in_features=1800, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=50, bias=True)
)

In [100]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         out_channels1 = 12
#         out_channels2 = 32
#         self.n_convs = 20
#         ## Warstwa konwolucyjna = ekstrakcja cech
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=out_channels1, kernel_size=5, stride=1, padding=0)
#         self.conv2 = nn.Conv2d(out_channels1, out_channels2, 5)

#         self.convs = [nn.Conv2d(in_channels=out_channels2, out_channels=out_channels2, kernel_size=5, stride=1, padding=2) for _ in range(self.n_convs)]
#         self.bns = [nn.BatchNorm2d(out_channels2) for _ in range(self.n_convs)]
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.pool2 = nn.MaxPool2d(2)
#         self.fc1 = nn.Linear(out_channels2 * 14 * 14, 2400)
#         self.fc2 = nn.Linear(2400, 600)
#         self.fc3 = nn.Linear(600, 50)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         for i in range(self.n_convs):
#             x = F.relu(self.convs[i](x))
#             x = self.bns[i](x)
#         x = self.pool1(x)
#         x = self.pool2(x)
#         x = torch.flatten(x, 1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [101]:
# net = Net().to(device)
# for conv in net.convs:
#     conv.to(device)
# for bn in net.bns:
#     bn.to(device)
# net

In [102]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [103]:
training_loss = 0
validation_loss = 0
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
  net.train()
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    
  net.eval()
  with torch.no_grad():
      for i, data in enumerate(validation_loader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          loss = criterion(outputs, labels) 
          validation_loss += loss.item()

  if epoch and (epoch % 1 == 0 or epoch == num_epochs - 1):    
      print(f"Epoch: {epoch}, training loss: {training_loss/1:.3}, validation loss: {validation_loss/1:.3}")
      training_loss = 0
      validation_loss = 0

In [ ]:
torch.save(net.state_dict(), 'model.tar.gz')

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
acc_sum = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels) 
        validation_loss += loss.item()
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
          if label == prediction:
              correct_pred[classes[label]] += 1
          total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    acc_sum += accuracy

    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

In [ ]:
acc_average_all_classes = acc_sum / len(classes)
acc_average_all_classes